In [1]:
_gmv_keys = [
    'temperature.output', 
    'temperature.man', 
    'temperature.interlocked', 
    'temperature.pv', 
    'temperature.sp', 
    'temperature.error', 
    'temperature.mode', 
    'agitation.output', 
    'agitation.man', 
    'agitation.interlocked', 
    'agitation.pv', 
    'agitation.sp', 
    'agitation.error', 
    'agitation.mode', 
    'ph.manUp',
    'ph.outputDown',
    'ph.error',
    'ph.pv',
    'ph.sp',
    'ph.manDown',
    'ph.outputUp',
    'ph.mode',
    'ph.interlocked',
    'do.manUp',
    'do.outputDown',
    'do.error',
    'do.pv',
    'do.sp',
    'do.manDown',
    'do.outputUp',
    'do.mode',
    'do.interlocked',
    'maingas.error',
    'maingas.man',
    'maingas.mode',
    'maingas.pv',
    'maingas.interlocked',
    'MFCs.air',
    'MFCs.n2',
    'MFCs.o2',
    'MFCs.co2',
    'condenser.output',
    'condenser.man',
    'condenser.pv',
    'condenser.sp', 
    'condenser.error', 
    'condenser.mode',
    'pressure.error',
    'pressure.pv',
    'level.error',
    'level.pv'
]

In [ ]:
import threading
from hello.hello3 import open_hello
from datetime import datetime
import time


class HelloLoggerTask(threading.Thread):
    def __init__(self, ip="", name=None, vars=(), poll_interval=2, start_time=None, _data_lock=None):
        
        # Config
        self.ip = ip
        self.vars = self._parse_vars(vars)
        self._name = name
        self.poll_interval = 5
        
        
        # Runtime
        self._data_lock = _data_lock or threading.Lock()
        self._data = {}
        self._h = None
        self._should_stop = False
        self._last_timestamp = None
        self._start_time = start_time
        
    def stop(self):
        self._should_stop = True
        
    def run(self):
        
        # create some locals for brevity
        
        self._h = h = open_hello(ip)
        call_hello = self._call_hello

        if self._name is None:
            self._name = call_hello(h.reactorname)
            
        if self._start_time is None:
            self._start_time = datetime.now()
        start_time = self._start_time
            
        # unpack some values so they can't be modified 
        # at runtime by anyone evil or just dumb (hopefully)
        recorded = self.vars
        
        # strings that don't need re-computing
        last_request_string = name + ".Last Request.Time"
        elapsed_time_string = name + ".Elapsed Time.hr"
            
        while not self._should_stop:
            
            # An existential question - when did the query actually occur?
            # Did it occur as soon as the request was sent, after it was
            # recieved, or halfway between the two? The most likely scenario
            # for a delayed response is that our server software is choking 
            # for some reason, so use the timestamp immediately after return
            # as the best estimate. 
            
            next_run = time.time() + self.poll_interval
            mv = call_hello(h.gpmv)
            t = datetime.now()
            
            data = [
                (last_request_string, t),
                (elapsed_time_string, (start_time - t).total_seconds() / 3600)
            ]
            
            for k1, k2 in recorded:
                v = mv[k1][k2]
                data.append((k1, k2, v))
            
            with self._data_lock:
                self._data = data
                
            # Sleep chokes on negative numbers
            left = next_run - time.time()
            if left > 0:
                time.sleep(left)
                
    def get_data(self):
        
        # Deepcopy of self._data isn't needed here, because a) 
        # the object is flat, not nested (like getMainValues is)
        # and b) at each iteration loop, the entire data structure
        # is replaced, not modified. 
        
        with self._data_lock:            
            return self._name, self._data
        
    def get_data_unsafe(self):
        return self._name, self._data
        
    def _parse_vars(self, vars):
        seen = set()
        keys = []
        for k in vars:
            if k in seen:
                continue
            seen.add(k)
            try:
                k1, k2 = k.split(".")
            except ValueError:  # no ".", bad key
                raise ValueError("%r is an invalid key" % k) from None
            keys.append((k1, k2))
        return tuple(keys)
        
    def _call_hello(f, *args, **kw):
        while True:
            try:
                return f(*args, **kw)
            except NotLoggedInError:
                self._h.login()
            
            # XXX Handle connection errors
        
class HelloMultiLogger():
    def __init__(self, filename, sample_interval=10):
        self._tasks = []
        self._should_stop = False
        self._data_lock = threading.Lock()
        self._filename = filename
        
    def stop(self):
        self._should_stop = True
        for t in self._tasks:
            t.stop()
        
    def add_logger(self, ip, name=None, vars=(), poll_interval=2, start_time=None, start=False):
        if self.is_alive():
            raise ValueError("Can't add tasks to already-running data logger.")
        task = HelloLoggerTask(ip, name, vars, poll_interval, self._data_lock)
        if start:
            task.start()
        self._tasks.append(task)
        return task
    
    def start_all(self):
        for t in self._tasks:
            if not t.is_alive():
                t.start()
        self.start()
                
    def run(self):
        data = []
        while True:
            data.clear()
            with self._data_lock:
                for t in self._tasks:
                    data.append(t.get_data_unsafe())
                    
        
            
            